# Omdena  - Milan Chapter Agrifoods
## AI for Sustainable agri-food systems: use of Satellite Imagery
### Data Exploration for fruits in Italy 2006-2021
#### Author: Maria Fisher 


The main objective of this study is to have gather information about crop production in Italy for the period of 2006-2021. 

Crop dataset used in this study was downloaded from the Italian National Institute of Statistics (Istat).



In [ ]:
import warnings 
warnings.filterwarnings("ignore")

import os
import pandas as pd
pd.options.display.float_format = "{:.2f}".format
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns 
import scipy 
import sklearn
import geopandas as gpd
import pgeocode
import folium
import sys
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot

In [ ]:
fruits = pd.read_csv('fruit_final_2006-2021.csv',skipinitialspace=True)
fruits.head()

## Pre-processing dataset 

In [ ]:
def show_info(fruits):
    print('DATASET SHAPE: ', fruits.shape, '\n')
    print('-'*50)
    print('FEATURE DATA TYPES:')
    print(fruits.info())
    print('\n', '-'*50)
    print('NUMBER OF UNIQUE VALUES PER FEATURE:', '\n')
    print(fruits.nunique())
    print('\n', '-'*50)
    print('NULL VALUES PER FEATURE')
    print(fruits.isnull().sum())
show_info(fruits)

In [ ]:
fruits.describe()

## Total fruits and nuts production in 2006-2021

In [ ]:
plt.figure(figsize= (10,5))
sns.barplot(x= 'Year', y= 'production_tonnes',data = fruits,palette='coolwarm')
plt.title('Total fruits and nuts production 2006-2021')
plt.xlabel('Year')
plt.ylabel('Total production (tonnes)')
plt.show()



## Cereal production by Cities 

In [ ]:
fruits_region = fruits.groupby(by = fruits.City)['production_tonnes','City'].sum().reset_index().sort_values(by = 'production_tonnes', ascending = False).head(10)
fruits_region

In [ ]:
plt.figure(figsize= (7,5))
sns.barplot(x=fruits_region['production_tonnes'],y= fruits_region['City'], orient='h', palette='coolwarm');
plt.title('Total fruits production 2006-2021 by City')
plt.xlabel('Total production')
#plt.ylabel('Cities')
plt.show()

## fruits highest production 2006-2022

In [ ]:
print(fruits.Type_crop.max())
print(fruits.Type_crop.value_counts())
print(fruits.Type_crop.nunique())



Dataset shows there are 55 different types of fruits and nuts cultivated in Italy. The top most produced crops are White oranges, Peach, Pear, Apples, Apricot, Cherry, Plum, Kiwi, Nectarine, Hazelnut, Almond and Raspberry.


In [ ]:
fruits.describe().astype(int)

In [ ]:
plt.figure(figsize=(10,5))
fruits['Type_crop'].value_counts().plot.bar()
plt.title('Total fruits production 2006-2021 by crop')
plt.ylabel('Total production (tonnes)')
plt.show()


## Subseting data

In [ ]:
fruits_top10 = fruits.apply(lambda row: row[fruits['Type_crop'].isin(['peach', 'pear', 'apple', 'apricot', 'hazelnut',
                                                                    'plum', 'kiwi', 'almond', 'cherry in complex','nectarine'])])

fruits_top10.head()

In [ ]:
plt.figure(figsize=(10,5))
fruits_top10['Type_crop'].value_counts().plot.bar()
plt.title('Total fruits production 2006-2021 by crop')
plt.ylabel('Total production (tonnes)')
plt.show()


In [ ]:
Q1 = fruits_top10.quantile(0.25)
Q3 = fruits_top10.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
# Remove outliers
fruits_top10 = fruits_top10[~((fruits_top10 < (Q1 - 1.5 * IQR)) | (fruits_top10 > (Q3 + 1.5 * IQR))).any(axis=1)]
print(fruits_top10.shape)


In [ ]:
#Box plot and histplot (crop production distribuition)

plt.figure(figsize=(13,5))

plt.subplot(1,2,1)
sns.boxplot(data=fruits_top10, x='production_tonnes', y="Type_crop",  color = 'darkblue', dodge=False)

plt.title(f'Box Plot total fruits production ')

plt.subplot(1,2,2)
sns.histplot(data=fruits_top10, x= 'production_tonnes',  color = 'darkblue', kde=True, bins = 80)
plt.title(f'Distribution total fruits production')

plt.show()



In [ ]:
#Box plot and histplot of total area(ha)

plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
sns.boxplot(data=fruits_top10, x='Year', y="total_ha",  color = 'darkblue', dodge=False)

plt.title(f'Box Plot total area (ha)')

plt.subplot(1,2,2)
sns.histplot(data=fruits_top10, x= 'total_ha',  color = 'darkblue', kde=True, bins = 80)
plt.title(f'Distribution total total area (ha)')

plt.show()

In [ ]:
print(fruits_top10.total_ha.max())
print(fruits_top10.total_ha.min())
print(fruits_top10.total_ha.value_counts())
print(fruits_top10.total_ha.nunique())


In [ ]:
print(fruits_top10.Fertilizers_tonnes.max())


In [ ]:
fig, ax = plt.subplots(3,1, sharex=True, figsize=(8,11))
fig.autofmt_xdate()

fruits_top10['Year'] = (fruits_top10['Year']).astype(str)

# Total crop production
crops = fruits[['Year', 'production_tonnes']]
crops = crops.groupby('Year').mean()
sns.lineplot(data= crops, x="Year", y="production_tonnes", ax=ax[0])

# Total area
crops = fruits[['Year', 'total_ha']]
crops = crops.groupby('Year').mean()
sns.lineplot(data= crops, x="Year", y="total_ha", ax=ax[1])

# Ferlizers
fertilizers = fruits[['Year', 'Fertilizers_tonnes']]
fertilizers = fertilizers.groupby('Year').mean()

sns.lineplot(data= fertilizers, x="Year", y="Fertilizers_tonnes", ax=ax[2])




In [ ]:
#Crop harvested by year

plotcrop10 = sns.relplot(kind='line', data=fruits_top10, x='Year', 
                         y='production_tonnes', hue='Type_crop', style='Type_crop', aspect=1.75)


In [ ]:
#Fertilizers distributed by year

plotfertilizers10 = sns.relplot(kind='line', data=fruits_top10, x='Year', 
                         y='Fertilizers_tonnes', hue='Type_fertilizer', style='Type_fertilizer', aspect=1.75)

# Climate data analysis

|Param_Code|Param_Name|Resolution|Depth [m]|Units|
|:--------:|:--------:|:--------:|:-------:|:---:|
|GWETPROF|Profile Soil Moisture|0.5° x 0.625°|1.34 - 8.53|water-free [0], saturated soil[1]| 
|GWETTOP|Surface Soil Wetness|0.5° x 0.625°|0.00 - 0.05| water-free [0], saturated soil[1]|
|GWETROOT|Root Zone Soil Wetness|0.5° x 0.625°|0.10 - 1.00|water-free [0], saturated soil[1]|
|CLOUD_AMT|Cloud Amount|---|---|%|
|TS|Earth Skin Temperature|---|---|°C|
|PS|Surface Pressure|---|---|kPa|
|RH2M|Relative Humidity at 2 Meters|---|---|%|
|QV2M|Specific Humidity at 2 Meters|---|---|g/kg|
|PRECTOTCORR|Precipitation Corrected|---|---|mm/day|
|T2M_MAX|Temperature at 2 Meters Maximum|---|---|°C|
|T2M_MIN|Temperature at 2 Meters Minimum|---|---|°C|
|T2M_RANGE|Temperature at 2 Meters Range|---|---|°C|
|WS2M|Wind Speed at 2 Meters|---|---|m/s|


  

*Precipitation Corrected (PRECTOTCORR)- [The bias corrected average of total precipitation at the surface of the earth in water mass (includes water content in snow)]

*Source? https://power.larc.nasa.gov/#resources

In [ ]:

fig, ax = plt.subplots(3, 2, sharex=True, figsize=(20,10))

fig.suptitle('Climate in Italy 2006 - 2021')

fruits['Year'] = (fruits['Year']).astype(str)

# relative_humidity
sns.lineplot(data= fruits, x="Year", y="RH2M", ax=ax[0,0])

# precipitation
sns.lineplot(data= fruits, x="Year", y="PRECTOTCORR", ax=ax[0,1])

# temp_max
sns.lineplot(data= fruits, x="Year", y="T2M_MIN", ax=ax[1,0])

# temp_min
sns.lineplot(data= fruits, x="Year", y="T2M_MAX", ax=ax[1,1])

# surface_soil_wetness
sns.lineplot(data= fruits, x="Year", y="GWETTOP", ax=ax[2,0])

# root_zone_wetness
sns.lineplot(data= fruits, x="Year", y="GWETROOT", ax=ax[2,1])


In [ ]:
fruits

In [ ]:
fruits_model = fruits_top10.drop(columns =['Year','PS','TS','GWETTOP', 'QV2M','WS2M','T2M_RANGE','GWETPROF','CLOUD_AMT','PRECTOTCORR','lat', 'lon'  ])
fruits_model

In [ ]:
fruits_model = fruits_model.to_csv('fruits_model.csv', index=False)

# References


http://dati.istat.it

https://maps.princeton.edu/catalog/stanford-mn871sp9778

https://www.crea.gov.it/documents/68457/0/ITACONTA+2020_ENG+DEF+xweb+%281%29.pdf/95c6b30a-1e18-8e94-d4ac-ce884aef76e8?t=1619527317576

https://seaborn.pydata.org/generated/seaborn.relplot.html

https://www.statisticshowto.com/variance-inflation-factor/

https://statisticsbyjim.com/regression/multicollinearity-in-regression-analysis/

https://lost-stats.github.io/Presentation/Figures/heatmap_colored_correlation_matrix.html

https://plotly.com/python/box-plots/

https://pycaret.org/